In [5]:
# ============================================================================
# STEP 1: RESPONSE GENERATION
# ============================================================================

# 1. Install Libraries
!pip install -q -U transformers accelerate bitsandbytes kagglehub

import torch
import kagglehub
import pandas as pd
from kagglehub import KaggleDatasetAdapter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# ------------------------------------------------------------------
# 🔑 AUTHENTICATION (Your Snippet)
# ------------------------------------------------------------------
try:
    # We use the exact code you provided to get the token
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("✅ Successfully logged in to Hugging Face!")
except Exception as e:
    print(f"❌ Auth Failed: {e}")
    print("⚠️ Make sure you added 'HF_TOKEN' in the 'Add-ons -> Secrets' menu.")

# ------------------------------------------------------------------
# 📂 LOAD DATASET (Using KaggleHub)
# ------------------------------------------------------------------
print("⬇️ Downloading Dataset...")
try:
    # Corrected syntax for kagglehub
    df = kagglehub.load_dataset(
      KaggleDatasetAdapter.PANDAS,
      "girisankargopakumar/hallucination-detection-dataset-400",
      "queries_400.csv"  # Specific file we want
    )
    print(f"✅ Loaded {len(df)} queries from Kaggle Hub")
except Exception as e:
    print(f"⚠️ KaggleHub failed ({e}). Checking local input...")
    # Fallback: If you added the dataset via the 'Add Input' button
    import glob
    files = glob.glob("/kaggle/input/**/queries_400.csv", recursive=True)
    if files:
        df = pd.read_csv(files[0])
        print(f"✅ Loaded {len(df)} queries from local input")
    else:
        raise FileNotFoundError("❌ Could not find queries_400.csv. Please check the dataset source.")

# ------------------------------------------------------------------
# 🤖 LOAD MODEL & GENERATE
# ------------------------------------------------------------------
print("🚀 Loading LLaMA 3.1 Model...")
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    quantization_config=bnb_config, 
    device_map="auto"
)

print("⚡ Generating Responses...")
responses = []

for i, row in df.iterrows():
    # Format prompt as a chat to get the model's true behavior
    messages = [{"role": "user", "content": row['query_text']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        # temperature=0.0 ensures the answer is deterministic (not random)
        outputs = model.generate(**inputs, max_new_tokens=150, do_sample=False, temperature=0.0)
        
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Clean up: remove the prompt, keep only the answer
    answer = full_text.split("assistant\n\n")[-1].strip()
    
    responses.append(answer)
    
    if i % 50 == 0: 
        print(f"   Processed {i}/{len(df)}...")

# ------------------------------------------------------------------
# 💾 SAVE RESULTS
# ------------------------------------------------------------------
df['model_response'] = responses
output_file = "queries_with_responses.csv"
df.to_csv(output_file, index=False)

print("\n" + "="*50)
print(f"✅ DONE! Go to the 'Output' tab and download: {output_file}")
print("="*50)

✅ Successfully logged in to Hugging Face!
⬇️ Downloading Dataset...


/tmp/ipykernel_48/4184472367.py:35: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


✅ Loaded 400 queries from Kaggle Hub
🚀 Loading LLaMA 3.1 Model...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

   Processed 300/400...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

   Processed 350/400...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for


✅ DONE! Go to the 'Output' tab and download: queries_with_responses.csv
